# 下载并直接运行

In [1]:
#模型下载
from modelscope import snapshot_download
model_dir = snapshot_download('qwen/Qwen2-0.5B-Instruct-GPTQ-Int4')

2024-06-22 11:42:18,953 - modelscope - INFO - PyTorch version 2.3.1 Found.
2024-06-22 11:42:18,954 - modelscope - INFO - Loading ast index from /home/ubuntu/.cache/modelscope/ast_indexer
2024-06-22 11:42:19,011 - modelscope - INFO - No valid ast index found from /home/ubuntu/.cache/modelscope/ast_indexer, generating ast index from prebuilt!
2024-06-22 11:42:19,064 - modelscope - INFO - Loading done! Current index file version is 1.15.0, with md5 e9a35559cccf70bdad8675fad207a115 and a total number of 980 components indexed
Downloading: 100%|██████████| 1.23k/1.23k [00:00<00:00, 4.86kB/s]
Downloading: 100%|██████████| 73.0/73.0 [00:00<00:00, 312B/s]
Downloading: 100%|██████████| 242/242 [00:00<00:00, 937B/s]
Downloading: 100%|██████████| 11.1k/11.1k [00:00<00:00, 42.8kB/s]
Downloading: 100%|██████████| 1.59M/1.59M [00:00<00:00, 2.42MB/s]
Downloading: 100%|██████████| 698M/698M [00:05<00:00, 130MB/s]  
Downloading: 100%|██████████| 3.63k/3.63k [00:00<00:00, 14.7kB/s]
Downloading: 100%|███

In [2]:
from modelscope import AutoModelForCausalLM, AutoTokenizer
device = "cuda" # the device to load the model onto

model = AutoModelForCausalLM.from_pretrained(
    "qwen/Qwen2-0.5B-Instruct-GPTQ-Int4",
    torch_dtype="auto",
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained("qwen/Qwen2-0.5B-Instruct-GPTQ-Int4")

prompt = "Give me a short introduction to large language model."
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": prompt}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)
model_inputs = tokenizer([text], return_tensors="pt").to(device)

generated_ids = model.generate(
    model_inputs.input_ids,
    max_new_tokens=512
)
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
print(response)

CUDA extension not installed.
CUDA extension not installed.
/home/ubuntu/miniconda3/envs/qwen2/lib/python3.8/site-packages/transformers/modeling_utils.py:4481: FutureWarning: `_is_quantized_training_enabled` is going to be deprecated in transformers 4.39.0. Please use `model.hf_quantizer.is_trainable` instead
  warnings.warn(
Some weights of the model checkpoint at /home/ubuntu/.cache/modelscope/hub/qwen/Qwen2-0___5B-Instruct-GPTQ-Int4 were not used when initializing Qwen2ForCausalLM: ['model.layers.0.mlp.down_proj.bias', 'model.layers.0.mlp.gate_proj.bias', 'model.layers.0.mlp.up_proj.bias', 'model.layers.0.self_attn.o_proj.bias', 'model.layers.1.mlp.down_proj.bias', 'model.layers.1.mlp.gate_proj.bias', 'model.layers.1.mlp.up_proj.bias', 'model.layers.1.self_attn.o_proj.bias', 'model.layers.10.mlp.down_proj.bias', 'model.layers.10.mlp.gate_proj.bias', 'model.layers.10.mlp.up_proj.bias', 'model.layers.10.self_attn.o_proj.bias', 'model.layers.11.mlp.down_proj.bias', 'model.layers.11.mlp

A large language model is a type of machine learning algorithm that uses a large amount of data to learn from it. It is typically trained on a large corpus of text, which is then fed into the model to generate new text. The goal of a large language model is to be able to generate human-like language at a high level of accuracy and fluency. This can be achieved by training the model on a large corpus of text, which is then fed into the model to generate new text. The model's output can be used to answer questions about the world, suggest new ideas, and provide information on various topics. Large language models are an important part of many applications in artificial intelligence, including chatbots, natural language processing, and recommendation systems.


# vLLM


In [3]:
from vllm import LLM, SamplingParams

In [4]:
prompts = [
    "Hello, my name is",
    "The president of the United States is",
    "The capital of France is",
    "The future of AI is",
]
sampling_params = SamplingParams(temperature=0.8, top_p=0.95)

In [5]:
llm = LLM(model="/home/ubuntu/.cache/modelscope/hub/qwen/Qwen2-0___5B-Instruct-GPTQ-Int4",dtype='half', gpu_memory_utilization=.5)

WARNING 06-22 12:14:39 config.py:217] gptq quantization is not fully optimized yet. The speed can be slower than non-quantized models.
INFO 06-22 12:14:39 llm_engine.py:161] Initializing an LLM engine (v0.5.0.post1) with config: model='/home/ubuntu/.cache/modelscope/hub/qwen/Qwen2-0___5B-Instruct-GPTQ-Int4', speculative_config=None, tokenizer='/home/ubuntu/.cache/modelscope/hub/qwen/Qwen2-0___5B-Instruct-GPTQ-Int4', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=32768, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, disable_custom_all_reduce=False, quantization=gptq, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), seed=0, served_model_name=/home/ubuntu/.cache/modelscope/hub/qwen/Qwen2-0___5B-Instruct-GPTQ-Int4)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


INFO 06-22 12:14:39 selector.py:131] Cannot use FlashAttention-2 backend for Volta and Turing GPUs.
INFO 06-22 12:14:39 selector.py:51] Using XFormers backend.
INFO 06-22 12:14:40 selector.py:131] Cannot use FlashAttention-2 backend for Volta and Turing GPUs.
INFO 06-22 12:14:40 selector.py:51] Using XFormers backend.
INFO 06-22 12:14:41 model_runner.py:160] Loading model weights took 0.4318 GB
INFO 06-22 12:14:43 gpu_executor.py:83] # GPU blocks: 24881, # CPU blocks: 21845
INFO 06-22 12:14:46 model_runner.py:889] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 06-22 12:14:46 model_runner.py:893] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
INFO

In [6]:
outputs = llm.generate(prompts, sampling_params)

# Print the outputs.
for output in outputs:
    prompt = output.prompt
    generated_text = output.outputs[0].text
    print(f"Prompt: {prompt!r}, Generated text: {generated_text!r}")

Processed prompts: 100%|██████████| 4/4 [00:00<00:00, 16.47it/s, est. speed input: 90.67 toks/s, output: 263.73 toks/s]

Prompt: 'Hello, my name is', Generated text: ' Jamie, and I am a 21 year old American with a degree in'
Prompt: 'The president of the United States is', Generated text: ' your President.\nThe President of the United States is a member of the executive branch'
Prompt: 'The capital of France is', Generated text: ' ________.\nA. Paris\nB. Rome\nC. Madrid\nD'
Prompt: 'The future of AI is', Generated text: ' a little blurry.\n1. AI is currently being developed in fields such as computer'


In [1]:

from openai import OpenAI

# Modify OpenAI's API key and API base to use vLLM's API server.
openai_api_key = "EMPTY"
openai_api_base = "http://localhost:8080/v1"
client = OpenAI(
    api_key=openai_api_key,
    base_url=openai_api_base,
)
completion = client.completions.create(model="/home/ubuntu/.cache/modelscope/hub/qwen/Qwen2-0___5B-Instruct-GPTQ-Int4",
                                      prompt="San Francisco is a")
print("Completion result:", completion)

Completion result: Completion(id='cmpl-b02d9fa51f24479d88e05eb86f453479', choices=[CompletionChoice(finish_reason='length', index=0, logprobs=None, text=' particularly deadly city in America to drive in, whether rumbling like a freight train', stop_reason=None)], created=1719059633, model='/home/ubuntu/.cache/modelscope/hub/qwen/Qwen2-0___5B-Instruct-GPTQ-Int4', object='text_completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=16, prompt_tokens=4, total_tokens=20))


In [3]:

from openai import OpenAI
# Set OpenAI's API key and API base to use vLLM's API server.
openai_api_key = "EMPTY"
openai_api_base = "http://localhost:8080/v1"

client = OpenAI(
    api_key=openai_api_key,
    base_url=openai_api_base,
)

chat_response = client.chat.completions.create(
    model="/home/ubuntu/.cache/modelscope/hub/qwen/Qwen2-0___5B-Instruct-GPTQ-Int4",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Tell me a joke."},
    ]
)
print("Chat response:", chat_response)


Chat response: ChatCompletion(id='cmpl-90e5f537d4604ae488a7088a24d66aaa', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="Sure, here's one for you! Why did the tomato turn red?\n\nBecause it saw the salad dressing!", role='assistant', function_call=None, tool_calls=[]), stop_reason=None)], created=1719059734, model='/home/ubuntu/.cache/modelscope/hub/qwen/Qwen2-0___5B-Instruct-GPTQ-Int4', object='chat.completion', service_tier=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=23, prompt_tokens=24, total_tokens=47))


# vLLM Deploying with Docker

In [ ]:
# 见README